## Code Description:

This code will use the downloaded Fermi LAT lightcurves from the LC repository to find number of alerts per year based on flux limits.

YOU NEED TO RUN fermi_data_initial_sample_selection BEFORE THIS!!


In [1]:
import os,sys
import numpy as np
import pandas as pd
import pyLCR
from astropy.io import fits
import pickle
import datetime
from codes_from_pylcr import *

import joblib
import glob
import matplotlib.pyplot as plt

# For training:
import xgboost as xgb




The Fermi-LAT Light Curve Repository Toolkit v0.1.0
Support Contact: Daniel Kocevski (daniel.kocevski@nasa.gov)


In [2]:
os.chdir('/Users/aadesai1/Desktop/In_use/ML_work/Fermi_amego_alert_project/main/Fermi_Sample/')

### Note that this code uses the Fermi LC saved as a pickle file in the following format:
  [array1],[array2],[array3],[array4]
-  array1 = [timebins_detections,flux] --------------> xvals = timebins_detections
-  array2 = [x_errors,np.transpose(flux_error)]------> xvals = timebins_detections
-  array3 = [timebins_upperlimits,flux_upper_limit]--> xvals = timebins_upperlimits
-  array4 = [timebins,ts] ----------------------------> xvals = timebins
-  array5 = indices for detections only!

In [3]:
saved_fermi_lcs = glob.glob(f'../../data_fermi_LC_from_lc_repo/*.pickle')

In [4]:
all_info = []
src_names = []
for indi_i_val in range(len(saved_fermi_lcs)):
    with open(saved_fermi_lcs[indi_i_val], 'rb') as handle:
        loaded_fermi_lc = pickle.load(handle)

    
    det_times=loaded_fermi_lc[0][0]
    det_flux_no_ul=loaded_fermi_lc[0][1]
    det_indices = loaded_fermi_lc[4]
    all_info.append([det_times,det_flux_no_ul,det_indices])
    src_names.append(saved_fermi_lcs[indi_i_val][-24:-7])

Now for each source we find a baseline flux. To find this we can get the minimum flux from each source. 

After that an assumption is taken that minimum flux times (some factor) constitues an alert

In [5]:
min_flux_values = []
for indi_arr in all_info:
    min_flux_values.append(np.amin(indi_arr[1]))

In [6]:
flux_factor_for_alert = 1 # If kept at 1 all fluxes will be used, as this is flux factor to minimum, and not average
#flux_factor_for_alert = 50

In [7]:
time_bins_yearly = [54466, #Jan 2008
                    54832, #Jan 2009 (leap)
                    55197, #Jan 2010
                    55562, #Jan 2011
                    55927, #Jan 2012
                    56293, #Jan 2013 (leap)
                    56658, #Jan 2014
                    57023, #Jan 2015
                    57388, #Jan 2016
                    57754, #Jan 2017 (leap)
                    58119, #Jan 2018
                    58484, #Jan 2019
                    58849, #Jan 2020
                    59215, #Jan 2021 (leap)
                    59580, #Jan 2022
                    59945, #Jan 2023
                    60310, #Jan 2024
                    60676, #Jan 2025 (leap)
                    61041] #Jan 2026

print(f'There are total number of {len(time_bins_yearly)-1} year bins')                

There are total number of 18 year bins


In [8]:
all_times = np.arange(54466,61050,30)  # (61041-54466)/500 = 13.15 

In [9]:
alerts_per_bin = []
num_alerts_per_bin = []
for src_index in range(len(src_names)):
    alerts_per_indi_bin=[]
    num_alerts_per_indi_bin=[]
    times=[]
    fluxes=[]
    indices=[]
    bin_used = []
    
    for mjd_index in range(len(time_bins_yearly)):
        if mjd_index==len(time_bins_yearly)-1:
            continue
        tmin = time_bins_yearly[mjd_index]
        tmax = time_bins_yearly[mjd_index+1]
        allowed_indices = (all_info[src_index][0] > tmin) & (all_info[src_index][0] < tmax) & (all_info[src_index][1] > flux_factor_for_alert*min_flux_values[src_index])
        for indi_time in all_info[src_index][0][allowed_indices]:
            times.append(indi_time)
        for indi_flux in all_info[src_index][1][allowed_indices]:
            fluxes.append(indi_flux)
        bin_value=0
        for indi_index in all_info[src_index][2][allowed_indices]:
            indices.append(indi_index)
            bin_value+=1
        for indi_index in all_info[src_index][2][allowed_indices]:
            bin_used.append(bin_value)
            
        num_alerts_per_indi_bin.append(len(all_info[src_index][0][allowed_indices]))
    alerts_per_bin.append([times,fluxes,indices])
    num_alerts_per_bin.append(num_alerts_per_indi_bin)
    

In [10]:
# Check 1 : Lets see the num alerts per time bin
"""
xvals = np.arange(1,19,1)
for src_index in range(len(src_names)):
    fig = plt.figure(figsize=(5,5))
    plt.plot(xvals,num_alerts_per_bin[src_index])
    plt.show()
    plt.close()
"""    

'\nxvals = np.arange(1,19,1)\nfor src_index in range(len(src_names)):\n    fig = plt.figure(figsize=(5,5))\n    plt.plot(xvals,num_alerts_per_bin[src_index])\n    plt.show()\n    plt.close()\n'

In [11]:
t_append = []
flux_append=[]
index_append=[]
for t_index in range(len(all_times)-1):
    tmin = all_times[t_index]
    tmax = all_times[t_index+1]
    t_append.append((tmax+tmin)/2)
for src_index in range(len(src_names)):
    flux_per_src=[]
    index_per_src=[]
    for t_index in range(len(all_times)-1):
        tmin = all_times[t_index]
        tmax = all_times[t_index+1]
        allowed_indices = (all_info[src_index][0] > tmin) & (all_info[src_index][0] < tmax) & (all_info[src_index][1] > flux_factor_for_alert*min_flux_values[src_index])
        if np.isnan(np.mean(all_info[src_index][1][allowed_indices]))==True:
            flux_per_src.append(0.0)
        else:
            flux_per_src.append(np.mean(all_info[src_index][1][allowed_indices]))
        if np.isnan(np.mean(all_info[src_index][2][allowed_indices]))==True:
            index_per_src.append(0.0)
        else:
            index_per_src.append(np.mean(all_info[src_index][2][allowed_indices]))
    flux_append.append(flux_per_src)
    index_append.append(index_per_src)
    

/Users/aadesai1/miniforge3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/aadesai1/miniforge3/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [12]:
count = 0
for a in flux_append[0]:
    if a!=0.0:
        count+=1
count

81

In [13]:
len(alerts_per_bin[0][0])

110

In [14]:
if os.path.isdir(f'alerts_per_year_flux_factor_{flux_factor_for_alert}')!=True:
    os.mkdir(f'alerts_per_year_flux_factor_{flux_factor_for_alert}')

for src_index in range(len(src_names)):
    save_file = f'alerts_per_year_flux_factor_{flux_factor_for_alert}/num_alerts_per_year_file_{src_names[src_index]}.npy'
    np.save(save_file,num_alerts_per_bin[src_index])
    # this one saves a array with only data
    save_file2 = f'alerts_per_year_flux_factor_{flux_factor_for_alert}/alerts_flux_data_per_year_file_{src_names[src_index]}.npy'
    np.save(save_file2,alerts_per_bin[src_index])
    # this one saves a array giving 0.0 value to times where there is no flux data
    #save_file2 = f'alerts_per_year_flux_factor_{flux_factor_for_alert}/alerts_flux_data_per_year_file_{src_names[src_index]}.npy'
    #np.save(save_file2,[t_append,flux_append[src_index],index_append[src_index]])
    
    